In [30]:
import Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using AtomsBuilder
using ComponentArrays
using DFTK
using ForwardDiff
using LinearAlgebra
using Plots
using PseudoPotentialData
using Random
using Unitful
using UnitfulAtomic

  Activating project at `~/git/tutorial-cecam-workshop-dftk-2025`


In [51]:
# system = bulk(:Si)
system = bulk(:Si; a=(0.94) * 5.46u"angstrom")
pseudopotentials = PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf")
(; lattice, atoms, positions) = DFTK.parse_system(system, pseudopotentials)
model = model_DFT(system; pseudopotentials, functionals=PBE())

kgrid = [4, 4, 4]
Ecut = 5  # a crude cutoff
basis = PlaneWaveBasis(model; Ecut, kgrid)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 6
    num. fft   threads   : 1

    Ecut                 : 5.0 Ha
    fft_size             : (16, 16, 16), 4096 total points
    kgrid                : MonkhorstPack([4, 4, 4])
    num.   red. kpoints  : 64
    num. irred. kpoints  : 8

    Discretized Model(gga_x_pbe+gga_c_pbe, 3D):
        lattice (in Bohr)    : [0         , 4.84942   , 4.84942   ]
                               [4.84942   , 0         , 4.84942   ]
                               [4.84942   , 4.84942   , 0         ]
        unit cell volume     : 228.09 Bohr³
    
        atoms                : Si₂
        pseudopot. family    : PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf")
    
        num. electrons       : 8
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
      

In [52]:
scfres = @time self_consistent_field(basis; tol=1e-8)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.394942802763                   -0.88    4.4   41.0ms
  2   -8.397848623227       -2.54       -1.70    1.0   22.4ms
  3   -8.398010409547       -3.79       -2.85    1.2   68.1ms
  4   -8.398038370333       -4.55       -2.98    2.8   34.6ms
  5   -8.398038712224       -6.47       -3.26    1.0   22.2ms
  6   -8.398038814580       -6.99       -5.00    1.0   21.9ms
  7   -8.398038818611       -8.39       -4.51    3.1   67.2ms
  8   -8.398038818900       -9.54       -5.34    1.4   26.2ms
  9   -8.398038818911      -10.95       -6.55    1.1   25.0ms
 10   -8.398038818912      -12.00       -6.64    2.5    228ms
 11   -8.398038818912      -13.97       -6.97    1.0   11.8ms
 12   -8.398038818912      -14.45       -8.39    1.0   12.9ms
  0.587609 seconds (430.61 k allocations: 171.753 MiB, 15.43% gc time, 32.14% compilation time)


(ham = Hamiltonian(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = MonkhorstPack([4, 4, 4])), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = MonkhorstPack([4, 4, 4])), KPoint([     0,      0,      0], spin = 1, num. G vectors =   113), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = MonkhorstPack([4, 4, 4])), KPoint([     0,      0,      0], spin = 1, num. G vectors =   113), [0.0, 0.6295239670415469, 2.5180958681661876, 2.5180958681661876, 0.6295239670415469, 0.6295239670415469, 0.8393652893887292, 2.308254545819005, 3.986985124596463, 1.6787305787774585  …  2.308254545819005, 4.196826446943646, 4.1968264469436445, 2.308254545819005, 1.6787305787774582, 0.8393652893887292, 2.308254545819005, 3.986985124596463, 1.6787305787774582, 

In [53]:
stress = compute_stresses_cart(scfres)

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.000561554   0.0           0.0
  0.0          -0.000561554   0.0
  0.0           0.0          -0.000561554

In [54]:
basis_ref = PlaneWaveBasis(model; Ecut=30, kgrid)
scfres_ref = @time self_consistent_field(basis_ref; tol=1e-8)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.433260213931                   -0.88    6.0    420ms
  2   -8.436814959745       -2.45       -1.77    1.0    315ms
  3   -8.437077796708       -3.58       -2.70    2.2    1.14s
  4   -8.437086769153       -5.05       -3.49    2.5    469ms
  5   -8.437086921827       -6.82       -4.55    2.2    448ms
  6   -8.437086925467       -8.44       -5.32    3.0    531ms
  7   -8.437086925486      -10.74       -6.12    2.1    311ms
  8   -8.437086925486      -12.27       -6.95    2.4    208ms
  9   -8.437086925486      -13.85       -7.81    2.4    202ms
 10   -8.437086925486   +  -14.75       -8.56    2.4    206ms
  4.301439 seconds (271.38 k allocations: 1.419 GiB, 18.17% gc time)


(ham = Hamiltonian(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :none), Ecut = 30.0 Ha, kgrid = MonkhorstPack([4, 4, 4])), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :none), Ecut = 30.0 Ha, kgrid = MonkhorstPack([4, 4, 4])), KPoint([     0,      0,      0], spin = 1, num. G vectors =  1807), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :none), Ecut = 30.0 Ha, kgrid = MonkhorstPack([4, 4, 4])), KPoint([     0,      0,      0], spin = 1, num. G vectors =  1807), [0.0, 0.6295239670415469, 2.5180958681661876, 5.665715703373921, 10.07238347266475, 15.738099176038673, 22.662862813495686, 22.662862813495686, 15.738099176038673, 10.07238347266475  …  14.269209919608398, 20.774290912371047, 28.538419839216793, 25.81048264870342, 18.46603636655204, 12.380638018483756, 7.554287604498564, 3.986985124596463, 1.678730578777458

In [55]:
stress_ref = compute_stresses_cart(scfres_ref)

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.000893085  -1.12938e-21   0.0
 -1.12938e-21  -0.000893085  -1.12938e-21
  0.0          -1.12938e-21  -0.000893085

In [56]:
refinement = refine_scfres(scfres, basis_ref; tol=1e-8)

n     log10(Residual norm)   Δtime 
---   --------------------   ------
  1                  -1.20   8.12ms
  2                  -1.46   7.96ms
  3                  -1.76   13.3ms
  4                  -2.04   10.8ms
  5                  -2.30   8.74ms
  6                  -2.55   8.23ms
  7                  -2.80   11.5ms
  8                  -3.11   8.49ms
  9                  -3.42   8.14ms
 10                  -3.76   8.12ms
 11                  -4.13   8.17ms
 12                  -4.38   10.7ms
 13                  -4.61   8.65ms
 14                  -4.80   8.25ms
 15                  -5.05   8.31ms
 16                  -5.27   8.22ms
 17                  -5.46   11.0ms
 18                  -5.70   8.36ms
 19                  -5.94   8.25ms
 20                  -6.21   8.13ms
 21                  -6.43   8.10ms
 22                  -6.73   11.2ms
 23                  -6.90   8.40ms
 24                  -7.20   8.20ms
 25                  -7.41   8.09ms
 26                  -7.64  

DFTK.RefinementResult{Float64}(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :none), Ecut = 30.0 Ha, kgrid = MonkhorstPack([4, 4, 4])), Matrix{ComplexF64}[[-0.05173780737667653 - 0.9641788269250035im 5.3271039378834225e-11 - 8.412364404134282e-11im -3.1865415278619855e-12 + 1.781053786254784e-11im -2.934999747228134e-11 + 1.32618636442866e-10im; -0.060096163710010815 - 0.05397510278870078im -0.10705669210730898 + 0.2845078096094558im 0.3454550735464722 - 0.14990474628375686im -0.12937497590852842 + 0.11197898276673567im; … ; 0.001074441439761327 + 0.02002314908663147im -0.029448277507167397 + 0.01334557783407293im 0.037254428447461935 + 0.014706623467469496im 0.03630279310396418 + 0.0026162937825686032im; -0.060096163598547345 - 0.05397510252758817im 4.1044391581780727e-7 + 4.4871076746189154e-7im -1.3851439399820209e-7 - 8.190453387176354e-7im 0.38812243899907184 - 0.33593841682589226im], [-0.5606745817513202 - 0.7740476727288195im -0.07427123200057573 - 0.0770

In [57]:
scfres_new = let 
    (; basis, ψ, δψ, occupation) = refinement
    ψ_new = [DFTK.ortho_qr(ψ[ik] + δψ[ik]) for ik = 1:length(basis.kpoints)]
    eigenvalues = scfres.eigenvalues
    εF = scfres.εF
    (; basis, ψ=ψ_new, occupation, eigenvalues, εF)
end

(basis = PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :none), Ecut = 30.0 Ha, kgrid = MonkhorstPack([4, 4, 4])), ψ = Matrix{ComplexF64}[[-0.05154414120091899 - 0.9605696895463939im -6.359918017576072e-13 - 7.899064323642346e-12im 6.693617882191916e-12 + 8.197511679880476e-13im 1.2240054803047684e-10 + 2.320753832163458e-11im; -0.06425882575781715 - 0.05771377935048238im -0.10727025056959594 + 0.28507534866757417im -0.346144190459376 + 0.15020377563975373im -0.1296330538445378 + 0.11220236115966503im; … ; 0.000888925904328557 + 0.016565903917223596im -0.030877058919638014 + 0.013993082896205851im -0.03906195163361895 - 0.015420164234029522im 0.03806414430715777 + 0.0027432314899436556im; -0.06425882568159146 - 0.05771377927232273im 4.103610212560846e-7 + 4.493891175835185e-7im 1.3926860274212327e-7 + 8.182323805521379e-7im 0.3888966699637152 - 0.33660854787634914im], [-0.5593543985392178 - 0.7722250740605495im 0.07446507526895521 + 0.07726990448999092im -2.31558

In [58]:
stress_refined = compute_stresses_cart(scfres_new)

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.000828442   0.0           0.0
  0.0          -0.000828442   0.0
  0.0           0.0          -0.000828442

In [59]:
relerr(x, y) = norm(x - y) / norm(y)

relerr (generic function with 1 method)

In [60]:
using DifferentiationInterface

In [61]:
s, δs = value_and_derivative(AutoForwardDiff(), 0.0) do ε
    T = typeof(ε)
    lattice = T.(refinement.basis.model.lattice)
    model = Model(refinement.basis.model; lattice)
    basis = PlaneWaveBasis(model, Ecut=refinement.basis.Ecut, kgrid=refinement.basis.kgrid)
    scfres_new = (; 
        basis,
        ψ=refinement.ψ .+ ε.*refinement.δψ,
        refinement.occupation,
        scfres.eigenvalues,
        scfres.εF,
    )
    compute_stresses_cart(scfres_new)
end

([-0.000561559702096872 -1.1293772630057337e-21 0.0; -1.1293772630057337e-21 -0.000561559702096872 -1.1293772630057337e-21; 0.0 -1.1293772630057337e-21 -0.0005615597020968717], [-0.00020172953761480512 0.0 0.0; 4.411629933616147e-24 -0.0002017295376148051 0.0; 0.0 4.411629933616147e-24 -0.0002017295376148051])

In [62]:
stress

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.000561554   0.0           0.0
  0.0          -0.000561554   0.0
  0.0           0.0          -0.000561554

In [63]:
s + δs

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.000763289  -1.12938e-21   0.0
 -1.12497e-21  -0.000763289  -1.12938e-21
  0.0          -1.12497e-21  -0.000763289

In [64]:
stress_refined

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.000828442   0.0           0.0
  0.0          -0.000828442   0.0
  0.0           0.0          -0.000828442

In [65]:
stress_ref

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.000893085  -1.12938e-21   0.0
 -1.12938e-21  -0.000893085  -1.12938e-21
  0.0          -1.12938e-21  -0.000893085

In [66]:
println("stress: \n", tr(stress))
println("Refined stress (Linearized)\n", tr(s + δs))
println("Refined stress:  \n", tr(stress_refined))
println("Reference stress:\n", tr(stress_ref))

println("Summary of relative errors:")
println("stress: ", relerr(tr(stress), tr(stress_ref)))
println("stress (linearized): ", relerr(tr(s + δs), tr(stress_ref)))
println("stress (refined): ", relerr(tr(stress_refined), tr(stress_ref)))

stress: 
-0.001684662065870542
Refined stress (Linearized)
-0.002289867719135031
Refined stress:  
-0.0024853258398848514
Reference stress:
-0.0026792557201850723
Summary of relative errors:
stress: 0.371220129090861
stress (linearized): 0.14533439197925602
stress (refined): 0.07238199729842326
